In [ ]:
def terminal_utility_tc(S, y):
    return np.array([utility(max(W + S * y - lambda_tc * S * abs(y), 1e-8)) for W in W_grid])

# ------------------------------
# Dynamic Programming with Transaction Costs (y-discrete)
# ------------------------------
def solve_dp_with_transaction_cost(option=False):
    Delta_y = 1.0
    M = 10
    y_grid = np.arange(-M, M+1) * Delta_y
    K_y = len(y_grid)

    U = {}
    for n in range(N_t + 1):
        U[n] = {}
        for i in range(n + 1):
            S_val = S0 * (u ** i) * (d ** (n - i))
            U[n][i] = np.zeros((K_y, N_W))
            for k, y in enumerate(y_grid):
                for l, W in enumerate(W_grid):
                    if option:
                        W_eff = W + S_val * y - lambda_tc * S_val * abs(y) - max(S_val - K, 0)
                    else:
                        W_eff = W + S_val * y - lambda_tc * S_val * abs(y)
                    W_eff = max(W_eff, 1e-8)
                    U[n][i][k, l] = utility(W_eff)

    for n in reversed(range(N_t)):
        for i in range(n + 1):
            S_val = S0 * (u ** i) * (d ** (n - i))
            for k, y in enumerate(y_grid):
                for l, W in enumerate(W_grid):
                    W_nt = W * np.exp(r * dt)
                    l_nt = np.argmin(np.abs(W_grid - W_nt))
                    val_nt = 0.5 * (U[n + 1][i + 1][k, l_nt] + U[n + 1][i][k, l_nt])
                    best_val = val_nt

                    # SELL
                    for m in range(1, M + 1):
                        k_new = k - m
                        if k_new < 0:
                            break
                        W_sell = W + (1 - lambda_tc) * S_val * m * Delta_y
                        if not (W_min <= W_sell <= W_max):
                            continue
                        l_s = np.argmin(np.abs(W_grid - W_sell))
                        if 1 <= k_new < K_y - 1 and 1 <= l_s < N_W - 1:
                            dU_dy = (U[n][i][k_new + 1, l_s] - U[n][i][k_new - 1, l_s]) / (2 * Delta_y)
                            dU_dW = (U[n][i][k_new, l_s + 1] - U[n][i][k_new, l_s - 1]) / (2 * (W_grid[1] - W_grid[0]))
                            if dU_dW != 0:
                                MRS = dU_dy / dU_dW
                                if MRS <= (1 + lambda_tc) * S_val:
                                    best_val = max(best_val, U[n][i][k_new, l_s])
                                    break

                    # BUY
                    for m in range(1, M + 1):
                        k_new = k + m
                        if k_new >= K_y:
                            break
                        W_buy = W - (1 + lambda_tc) * S_val * m * Delta_y
                        if not (W_min <= W_buy <= W_max):
                            continue
                        l_b = np.argmin(np.abs(W_grid - W_buy))
                        if 1 <= k_new < K_y - 1 and 1 <= l_b < N_W - 1:
                            dU_dy = (U[n][i][k_new + 1, l_b] - U[n][i][k_new - 1, l_b]) / (2 * Delta_y)
                            dU_dW = (U[n][i][k_new, l_b + 1] - U[n][i][k_new, l_b - 1]) / (2 * (W_grid[1] - W_grid[0]))
                            if dU_dW != 0:
                                MRS = dU_dy / dU_dW
                                if MRS >= (1 - lambda_tc) * S_val:
                                    best_val = max(best_val, U[n][i][k_new, l_b])
                                    break

                    U[n][i][k, l] = best_val
    return U, y_grid

# ------------------------------
# Solve and Plot Transaction Cost Case (no option)
# ------------------------------
U_tc, y_grid = solve_dp_with_transaction_cost(option=False)
k0 = len(y_grid) // 2

plt.figure(figsize=(10, 6))
plt.plot(W_grid, U_tc[0][0][k0], label="U(0, S0, y=0, W)")
plt.xlabel("Wealth, W")
plt.ylabel("Value Function U")
plt.title("Indirect Utility Function at t=0 with Transaction Costs (y=0)")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def terminal_utility_tc_option(S, y):
    payoff = max(S - K, 0.0)
    return np.array([utility(max(W + S * y - lambda_tc * S * abs(y) - payoff, 1e-8)) for W in W_grid])

# ------------------------------
# Solve and Plot Transaction Cost Case (option)
# ------------------------------
U_tc_opt, y_grid_opt = solve_dp_with_transaction_cost(option=True)
k0 = len(y_grid) // 2

plt.figure(figsize=(10, 6))
plt.plot(W_grid, U_tc_opt[0][0][k0], label="U(0, S0, y=0, W)")
plt.xlabel("Wealth, W")
plt.ylabel("Value Function U")
plt.title("Indirect Utility Function at t=0 with Transaction Costs (y=0)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ------------------------------
# Indifference Pricing with Transaction Costs (Corrected Shift Direction)
# ------------------------------

def shifted_utility_tc(C, U_tc_opt, U_tc, y_index):
    """
    Shift the with-option utility by C and compare to no-option utility.
    """
    W_shifted = W_grid + C
    U_shifted = np.interp(W_grid, W_shifted, U_tc_opt[0][0][y_index])
    diff = U_shifted - U_tc[0][0][y_index]
    return np.sqrt(np.mean(diff**2))  # RMSE

# Optimize to find C*
res_tc = minimize_scalar(
    lambda C: shifted_utility_tc(C, U_tc_opt, U_tc, k0),
    bounds=(0, 50),
    method='bounded'
)

C_star_tc = res_tc.x
print(f"Optimal indifference price with transaction costs (C*): {C_star_tc:.4f}")

# Plot comparison
W_shifted = W_grid + C_star_tc
U_shifted = np.interp(W_grid, W_shifted, U_tc_opt[0][0][k0])

plt.figure(figsize=(10, 6))
plt.plot(W_grid, U_tc[0][0][k0], 'b-', label="U without option")
plt.plot(W_grid, U_shifted, 'r--', label="U with option (shifted)")
plt.xlabel("Wealth, W")
plt.ylabel("Value Function U")
plt.title("Indifference Pricing with Transaction Costs (Corrected)")
plt.legend()
plt.grid(True)
plt.show()